In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import scipy
from tqdm.auto import tqdm, trange
import xgboost as xgb
import joblib
from sklearn.model_selection import cross_validate
import zipfile

from src.model import tscv, ClippedOutputRegressor
from src.feature_engineering import get_feature_cols, df_to_X_y, drop_non_features, add_lagged_features


%run constants.py

baseline_reg = joblib.load(os.path.join(MODELS_DIR, 'xgb-baseline.model'))

%matplotlib inline
print("Versions:")
print("  Python: %s" % sys.version)
for module in [pd, np, sns, sklearn]:
    print("  %s: %s" %(module.__name__, module.__version__))

Versions:
  Python: 3.8.2 (default, Jul 16 2020, 14:00:26) 
[GCC 9.3.0]
  pandas: 1.1.1
  numpy: 1.19.1
  seaborn: 0.10.1
  sklearn: 0.23.2


Now I want to focus on features relationships. We have quite a few of features so far, so we can try generating features by combining them.

The first one will be the price changes with relation to economics indicators.

In [2]:
train_set = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'train-set.parquet'))
economics_hist = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'economics-history.parquet'))
prices_stats = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'prices-statistics.parquet'))

In [3]:
aux_df = prices_stats.copy()
aux_df['date_block_num'] = aux_df['date_block_num'] + 1
prices_stats_gains = prices_stats.merge(aux_df, on=['item_id', 'shop_id', 'date_block_num'], how='left', sort=False, suffixes=['_now', '_prev'])
prices_stats_gains.head()

,item_id,shop_id,date_block_num,item_shop_price_median_now,cat_price_median_now,cat_shop_price_median_now,item_price_median_now,item_shop_price_median_prev,cat_price_median_prev,cat_shop_price_median_prev,item_price_median_prev
0,5037,5,10,1699.0,1699.0,1999.0,1699.0,NaN,NaN,NaN,NaN
1,5037,4,10,1699.0,1699.0,1649.0,1699.0,NaN,NaN,NaN,NaN
2,5037,6,10,1699.0,1699.0,1999.0,1699.0,NaN,NaN,NaN,NaN
3,5037,3,10,1699.0,1699.0,1499.0,1699.0,NaN,NaN,NaN,NaN
4,5037,2,10,1699.0,1699.0,1499.0,1699.0,NaN,NaN,NaN,NaN


In [5]:
prices_cols = ['item_shop_price_median', 'cat_price_median', 'cat_shop_price_median', 'item_price_median']
for col in prices_cols:
    prices_stats_gains['%s_gain' % col] = ((prices_stats_gains['%s_now' % col] - prices_stats_gains['%s_prev' % col]) / prices_stats_gains['%s_now' % col]).fillna(1.0)

In [6]:
prices_stats_gains.describe()

,item_id,shop_id,date_block_num,item_shop_price_median_now,cat_price_median_now,cat_shop_price_median_now,item_price_median_now,item_shop_price_median_prev,cat_price_median_prev,cat_shop_price_median_prev,item_price_median_prev,item_shop_price_median_gain,cat_price_median_gain,cat_shop_price_median_gain,item_price_median_gain
count,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,4.926600e+06,4.926600e+06,4.926600e+06,4.926600e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06
mean,1.101940e+04,3.164286e+01,2.150000e+01,7.171264e+02,7.033555e+02,7.112942e+02,7.850688e+02,7.097926e+02,6.964006e+02,7.050839e+02,7.764684e+02,-1.020204e-01,4.440177e-02,1.239136e-03,-2.565242e-01
std,6.252631e+03,1.756189e+01,6.922187e+00,1.516970e+03,1.497071e+03,1.506284e+03,1.560655e+03,1.482710e+03,1.463426e+03,1.475410e+03,1.529924e+03,1.989264e+01,2.291482e-01,1.439572e+00,2.951799e+01
min,3.000000e+01,2.000000e+00,1.000000e+01,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-9.990000e+02,-5.197000e+03,-7.109737e-01,-9.980000e+01,-5.197000e+03
25%,5.381500e+03,1.600000e+01,1.575000e+01,2.990000e+02,2.990000e+02,2.990000e+02,2.990000e+02,2.990000e+02,2.990000e+02,2.990000e+02,2.990000e+02,0.000000e+00,0.000000e+00,-7.296137e-02,0.000000e+00
50%,1.120300e+04,3.450000e+01,2.150000e+01,3.990000e+02,3.590000e+02,3.990000e+02,3.990000e+02,3.590000e+02,3.590000e+02,3.990000e+02,3.990000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.607150e+04,4.700000e+01,2.725000e+01,6.990000e+02,6.590000e+02,6.990000e+02,7.990000e+02,6.990000e+02,6.590000e+02,6.990000e+02,7.490000e+02,5.042017e-02,3.861004e-02,1.253133e-01,0.000000e+00
max,2.216700e+04,5.900000e+01,3.300000e+01,4.299000e+04,2.839200e+04,4.299000e+04,4.199000e+04,3.349000e+04,2.599000e+04,3.299000e+04,3.299000e+04,6.453057e+00,6.453057e+00,6.453057e+00,6.453057e+00


In [7]:
economics_hist_gains = economics_hist.copy()

econ_cols = ['CNYRUB', 'EURRUB', 'USDRUB', 'MOEX']
for col in econ_cols:
    economics_hist_gains['%s_gain' % col] =  ((economics_hist_gains['%s_close' % col] - economics_hist_gains['%s_open' % col]) / economics_hist_gains['%s_close' % col]).fillna(1)
economics_hist_gains.describe()

,date_block_num,CNYRUB_open,CNYRUB_close,EURRUB_open,EURRUB_close,USDRUB_open,USDRUB_close,MOEX_open,MOEX_high,MOEX_low,MOEX_close,CNYRUB_gain,EURRUB_gain,USDRUB_gain,MOEX_gain
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,33.000000,33.000000,33.000000,33.000000,35.000000,35.000000,35.000000,35.000000
mean,17.000000,6.912036,7.051546,52.920974,53.690800,42.858722,43.782481,1502.956667,1572.121515,1444.673333,1510.738788,0.017022,0.012547,0.017666,0.060733
std,10.246951,2.055718,2.108321,11.777598,11.940725,13.119323,13.555482,127.848997,142.924084,132.229980,136.920868,0.059836,0.057412,0.061565,0.238257
min,0.000000,4.814853,4.838964,39.761700,39.761700,29.983436,30.093358,1304.930000,1345.480000,1182.890000,1306.010000,-0.152468,-0.154986,-0.148345,-0.081830
25%,8.500000,5.386638,5.403534,43.684500,43.914500,32.946348,32.914444,1401.340000,1469.400000,1343.990000,1400.710000,-0.008252,-0.006852,-0.012579,-0.025385
50%,17.000000,5.794670,5.828424,48.442000,48.780000,35.714368,35.714368,1473.540000,1518.120000,1429.280000,1476.380000,0.010889,0.013194,0.017914,0.016192
75%,25.500000,8.764054,9.174595,61.976900,62.397500,54.326402,56.881987,1625.210000,1706.290000,1570.460000,1642.970000,0.041868,0.039173,0.040140,0.037459
max,34.000000,11.314571,11.314571,79.925000,79.925000,70.698806,70.698806,1759.250000,1873.530000,1701.250000,1771.050000,0.151615,0.122056,0.157259,1.000000


In [8]:
combo_gains = prices_stats_gains.merge(economics_hist_gains, on='date_block_num', how='left', sort=False).fillna(1)

In [9]:
combo_gains[[col for col in combo_gains.columns if col.endswith('_gain')]].describe()

,item_shop_price_median_gain,cat_price_median_gain,cat_shop_price_median_gain,item_price_median_gain,CNYRUB_gain,EURRUB_gain,USDRUB_gain,MOEX_gain
count,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06
mean,-1.020204e-01,4.440177e-02,1.239136e-03,-2.565242e-01,2.127412e-02,1.531425e-02,2.272461e-02,3.351139e-03
std,1.989264e+01,2.291482e-01,1.439572e+00,2.951799e+01,7.000779e-02,6.740996e-02,7.181589e-02,4.518638e-02
min,-5.197000e+03,-7.109737e-01,-9.980000e+01,-5.197000e+03,-1.524676e-01,-1.549855e-01,-1.483448e-01,-8.182981e-02
25%,0.000000e+00,0.000000e+00,-7.296137e-02,0.000000e+00,-1.684404e-02,-2.102955e-02,-2.221238e-02,-4.080483e-02
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.358764e-02,1.583342e-02,2.487894e-02,1.241884e-02
75%,5.042017e-02,3.861004e-02,1.253133e-01,0.000000e+00,6.308251e-02,6.757043e-02,7.199375e-02,3.742547e-02
max,6.453057e+00,6.453057e+00,6.453057e+00,6.453057e+00,1.516149e-01,1.220556e-01,1.572593e-01,8.875512e-02


Now let's combine them.

In [10]:
eps=1e-5
for col1 in prices_cols:
    for col2 in econ_cols:
        combo_gains['%s_%s_ratio' % (col1, col2)] = combo_gains["%s_gain" % col2] / (eps + combo_gains["%s_gain" % col1])

In [11]:
combo_gains[[col for col in combo_gains.columns if col.endswith('_ratio')]].describe()

,item_shop_price_median_CNYRUB_ratio,item_shop_price_median_EURRUB_ratio,item_shop_price_median_USDRUB_ratio,item_shop_price_median_MOEX_ratio,cat_price_median_CNYRUB_ratio,cat_price_median_EURRUB_ratio,cat_price_median_USDRUB_ratio,cat_price_median_MOEX_ratio,cat_shop_price_median_CNYRUB_ratio,cat_shop_price_median_EURRUB_ratio,cat_shop_price_median_USDRUB_ratio,cat_shop_price_median_MOEX_ratio,item_price_median_CNYRUB_ratio,item_price_median_EURRUB_ratio,item_price_median_USDRUB_ratio,item_price_median_MOEX_ratio
count,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06
mean,1.209329e+03,7.603219e+02,1.290475e+03,1.042615e+02,1.282672e+03,7.996290e+02,1.369355e+03,9.568334e+01,7.829300e+02,5.433781e+02,8.358754e+02,1.420852e+02,1.382058e+03,9.226540e+02,1.472624e+03,2.393324e+02
std,5.338264e+03,5.175086e+03,5.508567e+03,3.351850e+03,5.476001e+03,5.331673e+03,5.654321e+03,3.448416e+03,4.407906e+03,4.165654e+03,4.527423e+03,2.705737e+03,5.746051e+03,5.505024e+03,5.921249e+03,3.559434e+03
min,-2.839041e+04,-2.869779e+04,-2.937356e+04,-8.816421e+03,-1.524676e+04,-1.549855e+04,-1.483448e+04,-8.182981e+03,-1.524676e+04,-1.549855e+04,-1.483448e+04,-8.182981e+03,-1.524676e+04,-1.549855e+04,-1.483448e+04,-2.092386e+04
25%,-3.662868e-01,-3.321772e-01,-3.919688e-01,-6.384027e-01,-5.075127e-01,-4.349926e-01,-5.163403e-01,-1.487380e+00,-3.055762e-01,-3.094522e-01,-3.463319e-01,-2.664626e-01,-4.262676e-01,-4.349926e-01,-4.267191e-01,-6.531246e-01
50%,1.720154e-01,1.368554e-01,1.962821e-01,3.668978e-02,2.489838e-01,2.723728e-01,3.031264e-01,7.039656e-02,3.286104e-02,3.439673e-02,4.706361e-02,3.282013e-02,2.395958e-01,2.368885e-01,2.464190e-01,9.201127e-02
75%,1.951789e+03,1.574993e+03,2.293977e+03,1.619179e+03,3.679364e+03,1.591692e+03,3.112447e+03,1.619179e+03,1.794396e+00,1.858500e+00,1.831858e+00,9.543563e-01,3.949074e+03,2.066941e+03,3.547056e+03,2.688344e+03
max,2.590357e+04,1.651754e+04,2.572654e+04,8.875512e+03,1.516149e+04,1.220556e+04,1.572593e+04,8.875512e+03,1.516149e+04,1.220556e+04,1.572593e+04,8.875512e+03,5.044937e+04,5.128253e+04,4.908519e+04,8.875512e+03


Let's build a train set using the new features as lagged features and check how it goes with the baseline model.

First let's check how the gains go.

In [18]:
train_set_combo_gains = add_lagged_features(train_set, combo_gains, [c for c in combo_gains.columns if c.endswith('_gain')], max_lag=6, fill_value=1)
train_set_combo_gains.describe()

,item_id,shop_id,date_block_num,item_cnt_month,f__item_shop_price_median_gain_1,f__cat_price_median_gain_1,f__cat_shop_price_median_gain_1,f__item_price_median_gain_1,f__CNYRUB_gain_1,f__EURRUB_gain_1,...,f__USDRUB_gain_2,f__MOEX_gain_2,f__item_shop_price_median_gain_3,f__cat_price_median_gain_3,f__cat_shop_price_median_gain_3,f__item_price_median_gain_3,f__CNYRUB_gain_3,f__EURRUB_gain_3,f__USDRUB_gain_3,f__MOEX_gain_3
count,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,...,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06,5.140800e+06
mean,1.101940e+04,3.164286e+01,2.150000e+01,2.199702e-01,-6.232891e-02,8.168011e-02,4.191892e-02,-2.123943e-01,6.369110e-02,5.846026e-02,...,1.067443e-01,8.664679e-02,2.589300e-02,1.644863e-01,1.256323e-01,-1.196137e-01,1.441486e-01,1.371966e-01,1.448004e-01,1.267499e-01
std,6.252631e+03,1.756189e+01,6.922187e+00,1.113889e+00,1.988943e+01,2.973046e-01,1.452155e+00,2.951826e+01,2.072445e-01,2.073046e-01,...,2.785488e-01,2.788439e-01,1.986649e+01,3.881752e-01,1.472308e+00,2.951880e+01,3.307605e-01,3.323056e-01,3.307120e-01,3.328724e-01
min,3.000000e+01,2.000000e+00,1.000000e+01,0.000000e+00,-5.197000e+03,-6.937870e-01,-9.980000e+01,-5.197000e+03,-1.524676e-01,-1.549855e-01,...,-1.483448e-01,-8.182981e-02,-5.197000e+03,-6.937870e-01,-9.980000e+01,-5.197000e+03,-1.524676e-01,-1.549855e-01,-1.483448e-01,-8.182981e-02
25%,5.381500e+03,1.600000e+01,1.575000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-6.396588e-02,0.000000e+00,-6.126432e-03,-7.407411e-03,...,-1.257260e-02,-3.004562e-02,0.000000e+00,0.000000e+00,-1.957447e-02,0.000000e+00,-6.126432e-03,-7.407411e-03,-1.257260e-02,-3.004562e-02
50%,1.120300e+04,3.450000e+01,2.150000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.222552e-02,1.829316e-02,...,3.329752e-02,1.801681e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.814219e-02,2.753324e-02,3.329752e-02,1.801681e-02
75%,1.607150e+04,4.700000e+01,2.725000e+01,0.000000e+00,6.671114e-02,5.722461e-02,1.432665e-01,3.539823e-02,7.217984e-02,7.480448e-02,...,8.772002e-02,4.096290e-02,1.519199e-01,1.228070e-01,2.367942e-01,1.432665e-01,9.577566e-02,8.365995e-02,9.853186e-02,5.157805e-02
max,2.216700e+04,5.900000e+01,3.300000e+01,2.000000e+01,6.453057e+00,6.453057e+00,6.453057e+00,6.453057e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,6.453057e+00,6.453057e+00,6.453057e+00,6.453057e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [19]:
X_train, y_train = df_to_X_y(train_set_combo_gains)

scores = cross_validate(baseline_reg, X_train, y=y_train,
                        cv=tscv.split(train_set_combo_gains['date_block_num'], window=16),
                        verbose=2, scoring='neg_root_mean_squared_error')
scores

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=  10.6s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.6s remaining:    0.0s


[CV] ................................................. , total=  10.7s
[CV]  ................................................................
[CV] ................................................. , total=  10.4s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   31.7s finished


{'fit_time': array([8.78158283, 8.96164536, 8.83346486]),
 'score_time': array([1.81970167, 1.72876048, 1.55873203]),
 'test_score': array([-1.04456123, -1.20960406, -1.10802251])}

In [20]:
np.mean(scores['test_score']), np.std(scores['test_score'])

(-1.1207292669442335, 0.06797490087464367)

Well, that's worse than using either economics indicators or price medians directly. Because of this I'll stop this experiment and move on. I'll probably get back to it eventually because this is something I strongly believe should add information to our models.